# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [3]:
print(os.getcwd())

c:\Users\Windows 10 2\OneDrive\Alex practice\p2


In [4]:
target_file = "event_datafile_new1"

In [5]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process, usecols=['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    return dataframe

In [8]:
def extract_to_df():
    # create an empty dataframe to hold all the data
    extracted_data = pd.DataFrame(columns=['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    # process all csv files
    
    # first get current folder and subfolder
    filepath = 'C:\\Users\\Windows 10 2\\OneDrive\\Alex practice\\p2\\event_data'
    
    # create a loop to create a list of files and collect each filepath
    for root, dirs, file in os.walk(filepath):
        
        # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*.csv'))
    
        for csvfile in file_path_list:
            extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)
        
    return  extracted_data

In [ ]:
all_csv_data = extract_to_df()
all_csv_data.dropna(inplace=True)
all_csv_data.head()

In [10]:
all_csv_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6831 entries, 2 to 8070
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   artist         6831 non-null   object
 1   firstName      6831 non-null   object
 2   gender         6831 non-null   object
 3   itemInSession  6831 non-null   object
 4   lastName       6831 non-null   object
 5   length         6831 non-null   object
 6   level          6831 non-null   object
 7   location       6831 non-null   object
 8   sessionId      6831 non-null   object
 9   song           6831 non-null   object
 10  userId         6831 non-null   object
dtypes: object(11)
memory usage: 640.4+ KB


In [11]:
all_csv_data = all_csv_data.astype({
    'artist': str,
    'firstName': str,
    'gender': str,
    'itemInSession': int,
    'lastName': str,
    'length': float,
    'level': str,
    'location': str,
    'sessionId': int,
    'song': str,
    'userId': float
}, errors='raise')
all_csv_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6831 entries, 2 to 8070
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6831 non-null   object 
 1   firstName      6831 non-null   object 
 2   gender         6831 non-null   object 
 3   itemInSession  6831 non-null   int32  
 4   lastName       6831 non-null   object 
 5   length         6831 non-null   float64
 6   level          6831 non-null   object 
 7   location       6831 non-null   object 
 8   sessionId      6831 non-null   int32  
 9   song           6831 non-null   object 
 10  userId         6831 non-null   float64
dtypes: float64(2), int32(2), object(7)
memory usage: 587.0+ KB


In [22]:
def load_to_file(terget_file,data_to_load):
    data_to_load.to_csv(target_file,index = False, encoding = 'utf-8')

In [23]:
# load the extracted data into a new csv file
load_to_file(target_file,all_csv_data)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster()

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [15]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydb
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [16]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

In [17]:
all_csv_data.head()[['artist', 'song', 'length', 'sessionId', 'itemInSession']]

,artist,song,length,sessionId,itemInSession
2,Des'ree,You Gotta Be,246.30812,139,1
4,Mr Oizo,Flat 55,144.03873,139,3
5,Tamba Trio,Quem Quiser Encontrar O Amor,177.18812,139,4
6,The Mars Volta,Eriatarka,380.42077,139,5
7,Infected Mushroom,Becoming Insane,440.26730,139,6


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = 'CREATE TABLE IF NOT EXISTS song_details '
query += '(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY(sessionId, itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [29]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'C:\\Users\\Windows 10 2\\OneDrive\\Alex practice\\p2\\event_datafile_new1'

with open(file, encoding='utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_details (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [59]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    select * 
    from song_details
    where sessionId = %s and itemInSession = %s;
"""

try:
    result = session.execute(query,(338,4))
except Exception as e:
    print(e)

for row in result:
    print(row.artist, row.song, row.length, row.sessionid, row.iteminsession)  

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [51]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
    CREATE TABLE IF NOT EXISTS artist_details
    (userId float, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession ASC)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [52]:
query = "INSERT INTO artist_details (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

file = 'C:\\Users\\Windows 10 2\\OneDrive\\Alex practice\\p2\\event_datafile_new1'

with open(file, encoding='utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_details (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (float(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))

In [58]:
query = """
    select * 
    from artist_details
    where userId = %s and sessionId = %s;
    """

try:
    result = session.execute(query,(10,182))
except Exception as e:
    print(e)

for row in result:
    print(row.userid, row.sessionid, row.iteminsession, row.artist, row.song, row.firstname, row.lastname) #   , , 

10.0 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
10.0 182 1 Three Drives Greece 2000 Sylvie Cruz
10.0 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10.0 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [78]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = 'CREATE TABLE IF NOT EXISTS user_details '
query += '(userId float, firstName text, lastName text, song text, PRIMARY KEY (song))'

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [77]:
session.execute('drop table user_details')

In [79]:
file = 'C:\\Users\\Windows 10 2\\OneDrive\\Alex practice\\p2\\event_datafile_new1'

with open(file, encoding='utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_details (userId, firstName, lastName, song) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (float(line[10]), line[1], line[4], line[9]))

In [80]:
query = """
    select *
    from user_details
    where song = %s;
"""

try:
    result = session.execute(query,('All Hands Against His Own',))
except Exception as e:
    print(e)

for row in result:
    print(row.userid, row.firstname, row.lastname, row.song)

80.0 Tegan Levine All Hands Against His Own


### Drop the tables before closing out the sessions

In [81]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS song_details")
    session.execute("DROP TABLE IF EXISTS artist_details")
    session.execute("DROP TABLE IF EXISTS user_details")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [82]:
session.shutdown()
cluster.shutdown()